In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
def web_scrape(brand, url):
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)
    
    driver.get(url)
    
    while True:
        try:
            wait.until(
                EC.visibility_of_element_located((
                    By.XPATH, '//button[contains(@class, "button js-load-more ")]')))
            driver.find_element_by_xpath(
                '//button[contains(@class, "button js-load-more ")]').click()
        except Exception:
            break
    
    main_page = driver.page_source
    main_soup = BeautifulSoup(main_page, "lxml")
    
    prod_pages = []
    prod_names = []
    prod_prices = []
    prod_dp = []
    prod_images = []
    
    for item in main_soup.find_all('a', {'class': 'item-link'}):
        href = item.attrs['href']
        item_link = 'https://www2.hm.com' + href
        prod_pages.append(item_link)
        
        page = requests.get(item_link).content
        soup = BeautifulSoup(page, "lxml")

        name = soup.find('h1', 
                         {'class': 'primary product-item-headline'}).text.strip('\r\n\t\t\t\t\t\t\t  ')
        prod_names.append(name)

        price = soup.find('span', 
                          {'class': 'price-value'}).text.strip('\r\n                $')
        price = float(price)
        prod_prices.append(price)

        dp = soup.find('p', {'class': 'pdp-description-text'}).text
        prod_dp.append(dp)

        image = soup.find('div', 
                          {'class': 'product-detail-main-image-container'}).find('img').attrs['src']
        url = 'http:' + image
        prod_images.append(url)
    
    d = {'prod_page_link': prod_pages, 
         'image_link': prod_images,
         'name': prod_names, 
         'price': prod_prices, 
         'description': prod_dp}
    
    df = pd.DataFrame(d)
    df['brand'] = brand
    
    return df

In [ ]:
brand = 'H&M'
url = 'https://www2.hm.com/en_us/women/products/view-all.html'
df = web_scrape(brand, url)
df.to_csv('HM.csv', index=False)